# Advanced Python: Building Scalable Applications

## Module 1

#### Threads, Processes, Generators and Coroutines
 - Introduction to generators, coroutines, threads and processes.
 - Concurrency Vs Parallelism: Choosing generator/coroutines Vs Threads/Processes

#### Python ```threading``` module: a deep-dive
 - Python threading module API
 - Creating and managing threads.
 - An overview on threading module.
 - Using the Thread class and the Timer class.
 - Active threads Vs. Daemon threads.
 - Helper functions in the threading module.


#### Threads
A form of concurrency model that allows a program to perform multitasking by means of abstracting its execution by a series of threads.

Thread models:
  1. Native Threading / Kernel-supporting Threading / Light-Weight-Processes (LWPs) -> 1:1 threading model
     These are threads fully managed by the OS kernel (scheduling, etc...)
  2. User Threading / Green Threads / User-level Threading / Coroutines / Greenlets / Eventlets -> N:1 threading model.

The `threading` module in Python provides `Thread` that implements Native Threading

Threads in python (from `threading` module) lack true parallelism. Every thread in python *must* acquire the Global-Interpreter-Lock (GIL) in order to execute any instructions. After a stipulated timeline (sys.getswitchinterval()), the thread in execution checks if there are any waiters on the GIL and releases the lock when necessary.

NOTE:
  - Native Threading can support "preemptive multitasking" based on the OS implementation
  - User Threading can only support cooperative multitasking.
  


In [1]:
import sys
sys.getswitchinterval()

0.005

In [2]:
## Implementing concurrency using generators

In [4]:
def fib(n):
    a, b = 0, 1
    for _ in range(n):
        print(a)
        a, b = b, a + b

fib(10)


0
1
1
2
3
5
8
13
21
34


In [6]:
from time import sleep

def fib_list(n):
    series = [0, 1]
    for _ in range(2, n):
        series.append(series[-1] + series[-2])
        sleep(0.5)
    return series

for i in fib_list(10):
    print(i, i*i)


0 0
1 1
1 1
2 4
3 9
5 25
8 64
13 169
21 441
34 1156


In [7]:
from time import sleep

def fib_gen(n):
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b
        sleep(0.5)

for i in fib_gen(10):
    print(i, i*i)


0 0
1 1
1 1
2 4
3 9
5 25
8 64
13 169
21 441
34 1156


#### Concurrency using Generators

In [ ]:
r = range(10)
r

In [ ]:
for v in r:
    print(v)

In [ ]:
def testfn():
    print("Start of testfn function...")
    return 100
    print("Back inside testfn function...")

testfn()
testfn()

In [ ]:
def testfn():
    print("Start of testfn function...")
    yield 100
    print("Back inside testfn function...")

g = testfn()
g


In [ ]:
iter(g)

In [ ]:
a = {11, 22, 33, 44}
print(a, type(a))
for v in a:
    print(v)

In [ ]:
a = [11, 22, 33, 44]
print(a, type(a))
li = iter(a)
print(li)

next(li)

In [ ]:
next(li)

In [ ]:
a = [11, 22, 33, 44, 55]
for v in a:
    print(v)

# ----
iterator = iter(a)
try:
    while True:
        v = next(iterator)
        # Body of 'for-loop'
        print(v)
except StopIteration:
    pass


In [ ]:
def testfn():
    print("Start of testfn function...")
    yield 100
    print("Back inside testfn function...")
    yield "Hello"
    print("Back again inside testfn function...")
    yield
    print("Back one more time inside testfn function...")
    yield
    print("End of testfn")

g = testfn()
g


In [ ]:
for v in testfn():
    print("In for loop: v =", v)
    

In [ ]:
def testfn():
    print("Start of testfn...")
    yield 100
    print("Back inside testfn...")
    return 200

g = testfn()
g

In [ ]:
for v in testfn():
    print(v)

In [ ]:
def fib(n):
    a, b = 0, 1
    for _ in range(n):
        print(a, end=" ")
        a, b = b, a + b

fib(10) # 0 1 1 2 3 5 8 13 21 34

In [ ]:
def fib_list(n): # This is reusable - but not concurrent!
    series = [0, 1]
    for _ in range(n-2):
        series.append(series[-1] + series[-2])
    return series

fib_list(10) # [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

for i in fib_list(10):
    print(i, i*i)

In [ ]:
from time import sleep

def fib_list(n): # This is reusable - but not concurrent!
    series = [0, 1]
    for _ in range(n-2):
        sleep(1)
        series.append(series[-1] + series[-2])
    return series

#fib_list(10) # [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

print("Start...")
for i in fib_list(10):
    print(i, i*i)

In [ ]:
from time import sleep

def fib_gen(n): # This is reusable and concurrent!
    a, b = 0, 1
    for _ in range(n):
        sleep(1)
        yield a
        a, b = b, a + b

print("Start...")
for i in fib_gen(10):
    print(i, i*i)

In [8]:
from time import sleep

def foo(n):
    while n > 0:
        print("Running foo with n =", n)
        n -= 1
        sleep(0.5)

def bar(n):
    while n > 0:
        print("Running bar with n =", n)
        n -= 1
        sleep(0.5)

foo(10)
bar(10)

Running foo with n = 10
Running foo with n = 9
Running foo with n = 8
Running foo with n = 7
Running foo with n = 6
Running foo with n = 5
Running foo with n = 4
Running foo with n = 3
Running foo with n = 2
Running foo with n = 1
Running bar with n = 10
Running bar with n = 9
Running bar with n = 8
Running bar with n = 7
Running bar with n = 6
Running bar with n = 5
Running bar with n = 4
Running bar with n = 3
Running bar with n = 2
Running bar with n = 1


In [ ]:
from time import sleep

def foo(n):
    while n > 0:
        print("Running foo with n =", n)
        n -= 1
        yield
        sleep(0.5)

def bar(n):
    while n > 0:
        print("Running bar with n =", n)
        n -= 1
        yield
        sleep(0.5)

# TODO: Implement a scheduler to run foo and bar concurrently using their generators
for _ in zip(foo(10), bar(10)):
    pass

Running foo with n = 10
Running bar with n = 10
Running foo with n = 9
Running bar with n = 9
Running foo with n = 8
Running bar with n = 8
Running foo with n = 7
Running bar with n = 7
Running foo with n = 6
Running bar with n = 6
Running foo with n = 5
Running bar with n = 5
Running foo with n = 4
Running bar with n = 4
Running foo with n = 3
Running bar with n = 3
Running foo with n = 2
Running bar with n = 2
Running foo with n = 1
Running bar with n = 1


#### Coroutines
Coroutines - are units of execution within a program that allows asynchronous execution with the aid of an event loop


Python has a history of different implementation of coroutines:
  1. Twisted Matrix project
  2. The `gevent` framework
  3. The `eventlet` framework
  4. The `asyncio` framework (part of Python 3.4+ standard library) - heavily inspired from The Twisted Matrix project.

#### Use-cases for Generators and Coroutines
  - Use generator for implement concurrent stream processing pipelines 
    (equivalent of Producer-Consumer patterns / pipelines)
  - Use coroutines for multiplexing I/O operations in a concurrent manner 
    (Asynchronous I/O operations)

Both Generators and Coroutines are suitable for cooperative multitasking workflows.

Coroutines are Python's implementation of "Green-Threads" / "User-Threads" / N:1 Threading Model.

### Introduction to Threads in Python (the ```threading``` module)

Threads in Python (threading module) provide a means of implementing
"Preemptive Multitasking" by leveraging the OS provided mechanisms for managing threads

This is also known as "Native Threading" / "OS-level Threading" / "Kernel-supported Threading" / Light-Weight-Processes (LWPs) / 1:1 Threading Model

The ```threading``` module provides a portable high-level API abstraction for creating and managing threads that works for Windows / Linux / MacOS / Other mainstream OS platforms.

The initial interface of threading module was heavily inspired from the Java's threading library.

Thread in Python are "preemptive", concurrent, but NOT parallel by default.


In [11]:
from threading import Thread

Scheduling and context-switching between threads are managed by the OS kernel (process/scheduler subsystem). The CPU usage by the OS to preempt and context-switch to another task (threads / process) is known as "scheduler latency".


In [12]:
Thread?

Init signature:
Thread(
    group=None,
    target=None,
    name=None,
    args=(),
    kwargs=None,
    *,
    daemon=None,
    context=None,
)
Docstring:     
A class that represents a thread of control.

This class can be safely subclassed in a limited fashion. There are two ways
to specify the activity: by passing a callable object to the constructor, or
by overriding the run() method in a subclass.
Init docstring:
This constructor should always be called with keyword arguments. Arguments are:

*group* should be None; reserved for future extension when a ThreadGroup
class is implemented.

*target* is the callable object to be invoked by the run()
method. Defaults to None, meaning nothing is called.

*name* is the thread name. By default, a unique name is constructed of
the form "Thread-N" where N is a small decimal number.

*args* is a list or tuple of arguments for the target invocation. Defaults to ().

*kwargs* is a dictionary of keyword arguments for the target
invocation. Def

In [13]:
from threading import Thread

class MyThread(Thread):
    def run(self):
        print("Hello from MyThread!")

t = MyThread()
t.start()
t.join()

Hello from MyThread!


#### Exercise: 
 - Implement a ```joinall()``` function to wait for all threads to exit 

#### Homework:
 - Create a ```RunPeriodic``` class that allows scheduling functions to run periodically at different intervals (like UNIX cron-job) until stopped.
